In [1]:
from PIL import Image
import os

def resize_image(input_path, output_path, max_size):
    with Image.open(input_path) as img:
        # Get current dimensions
        original_width, original_height = img.size

        # Determine the scale factor, keeping aspect ratio
        if original_height > original_width:
            scale = max_size / float(original_height)
            new_height = max_size
            new_width = int(scale * original_width)
        else:
            scale = max_size / float(original_width)
            new_width = max_size
            new_height = int(scale * original_height)

        # Only resize if image is larger than max size
        if original_width > max_size or original_height > max_size:
            img = img.resize((new_width, new_height), Image.LANCZOS)

        # Save the image, possibly overwriting the original
        img.save(output_path)

# Set the directory path and maximum dimension
directory_path = r"D:\Projects\Weed Detection\resized data\Processed data"
max_dimension = 512

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.ico', '.webp','.tif','JPG')):
        # Create the full path to the image
        file_path = os.path.join(directory_path, filename)
        # Resize and save the image
        resize_image(file_path, file_path, max_dimension)

print("Resizing complete.")


Resizing complete.


In [2]:
from PIL import Image
import os

def resize_image(input_path, output_path, max_size):
  with Image.open(input_path) as img:
    # Get current dimensions
    original_width, original_height = img.size

    # Determine the scale factor, keeping aspect ratio
    if original_height > original_width:
      scale = max_size / float(original_height)
      new_height = max_size
      new_width = int(scale * original_width)
    else:
      scale = max_size / float(original_width)
      new_width = max_size
      new_height = int(scale * original_height)

    # Only resize if image is larger than max size
    if original_width > max_size or original_height > max_size:
      img = img.resize((new_width, new_height), Image.LANCZOS)

    # Save the image, possibly overwriting the original
    img.save(output_path)

def process_folder(folder_path, max_dimension):
  # Iterate over entries in the folder
  for entry in os.listdir(folder_path):
    # Construct full path
    full_path = os.path.join(folder_path, entry)

    # Check if it's a file with supported image extensions
    if os.path.isfile(full_path) and full_path.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.ico', '.webp','.tif','JPG')):
      # Resize the image
      resize_image(full_path, full_path, max_dimension)
    # Recursively process subfolders if it's a directory
    elif os.path.isdir(full_path):
      process_folder(full_path, max_dimension)

# Set the directory path and maximum dimension
directory_path = r"D:\Projects\Weed Detection\resized data\Processed data"
max_dimension = 512

# Start processing the main folder
process_folder(directory_path, max_dimension)

print("Resizing complete.")


Resizing complete.


In [58]:
import cv2
import numpy as np
import os

def detect_green_areas(input_dir, output_dir, min_area_fraction=0.001):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    counter = 1
    # Loop through every file in the input directory
    for filename in os.listdir(input_dir):

        if filename.endswith(('.png', '.jpg', '.jpeg','.tif','.JPG')):  # You can add more file extensions if needed
            image_path = os.path.join(input_dir, filename)
            
            # Read the image
            image = cv2.imread(image_path)

            # Convert image to HSV
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

            # Define the lower and upper bounds for green in HSV
            lower_violit_1 = np.array([110, 50, 5])  # Adjusted lower bound for green
            upper_violit_1 = np.array([179, 255, 255])  # Adjusted upper bound for green
            
            lower_violit_2 = np.array([0, 50, 50])  # Adjusted lower bound for green
            upper_violit_2 = np.array([12, 255, 255])  # Adjusted upper bound for green

            # Define the lower and upper bounds for green color
            lower_green = np.array([35, 30, 85])
            upper_green = np.array([90, 255, 255])

            # Create a binary mask where green regions are white
            green_mask = cv2.inRange(hsv, lower_green, upper_green)
            violit_mask_1 = cv2.inRange(hsv, lower_violit_1, upper_violit_1) 
            violit_mask_2 = cv2.inRange(hsv, lower_violit_2, upper_violit_2)

            # Combine green and brown_2 masks
            combined_mask = cv2.bitwise_or(green_mask, violit_mask_1)
            combined_mask = cv2.bitwise_or(combined_mask, violit_mask_2)
            
            # Morphological operations to remove noise
            kernel = np.ones((3, 3), np.uint8)
            combined_mask = cv2.erode(combined_mask, kernel, iterations=1)
            combined_mask = cv2.dilate(combined_mask, kernel, iterations=1)
            combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)
            combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)

            # Calculate minimum area based on image dimensions
            min_area = min_area_fraction * image.shape[0] * image.shape[1]

            # Find connected components and their statistics
            num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(combined_mask, connectivity=8)

            # Filter out components based on their area
            for i in range(1, num_labels):  # Exclude background label (0)
                if stats[i, cv2.CC_STAT_AREA] < min_area:
                    combined_mask[labels == i] = 0  # Set pixels of small components to 0

            # Create a white background image
            white_background = np.ones_like(image) * 255

            # Merge the original image and the white background using the mask
            result = cv2.bitwise_and(image, image, mask=combined_mask)
            result += cv2.bitwise_and(white_background, white_background, mask=cv2.bitwise_not(combined_mask))

            # Save the result to the output directory
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, result)
            print(counter)
            counter += 1

# Paths to the directories
input_directory = r"D:\Projects\Weed Detection\mohab data\to be done\Purslane\resized"
output_directory = r"D:\Projects\Weed Detection\mohab data\to be done\Purslane\processed"

# Call the function with min_area_fraction set to 0.001 (adjust as needed)
detect_green_areas(input_directory, output_directory, min_area_fraction=0.001)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
